# SANDAG Sector Estimates Analysis
This is a calculator that takes in EDD data and aggregates it to specified SANDAG sectors. This also takes in estimates data and aggregates those to SANDAG sectors as well. Then a comparison between the two is drawn. 

See Purva's Excel for more information: https://sandag.sharepoint.com/:x:/r/qaqc/_layouts/15/Doc.aspx?sourcedoc=%7BCFD80313-6A49-43E6-B776-50C85CD17DD4%7D&file=EDD_Forecast%20Output%20Industry%20Level%20Jobs%20Comparison_QA.xlsx&action=default&mobileredirect=true&cid=9c4c556c-a447-4a46-8e56-ab5f1c95311a

In [1]:
import pandas as pd
import urllib.request  # For downloading the xlsx file
import pandas as pd
from sodapy import Socrata
import ssl
import sqlalchemy

# Grabbing EDD Data

In [2]:
# Sector Level Data
client = Socrata("data.edd.ca.gov", None)
results = client.get_all("r4zm-kdcg", area_name='San Diego-Carlsbad MSA')
results_df = pd.DataFrame.from_records(results)

In [3]:
# Cleaning the sector Data
edd_data = results_df[results_df['seasonally_adjusted']=='N'][['year', 'month', 'industry_title', 'current_employment']]
edd_data['date'] = edd_data.assign(day=1)[['year','month','day']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
edd_data['date'] = pd.to_datetime(edd_data['date']) #year-month-day
edd_data = edd_data.pivot(index='industry_title', columns='date', values='current_employment')
edd_data = edd_data.apply(pd.to_numeric)

In [4]:
# Grab the EDD sectors that we are interested in 
edd_breakdown = ['Mining and Logging','Total Farm','Construction','Manufacturing','Wholesale Trade','Retail Trade','Utilities','Transportation and Warehousing','Information','Finance and Insurance','Real Estate and Rental and Leasing','Professional, Scientific and Technical S','Management of Companies and Enterprises','Administrative and Support and Waste Ser','Educational Services','Health Care and Social Assistance','Arts, Entertainment, and Recreation','Accommodation','Food Services and Drinking Places','Other Services','Federal Government excluding Department of Defense','Department of Defense','State Government Education','State Government Excluding Education','Local Government Education','Local Government Excluding Education']
edd_data = edd_data[edd_data.index.isin(edd_breakdown)]

In [5]:
edd_data = pd.DataFrame(edd_data.iloc[:,-1]) # Filter for the latest data
edd_data.columns = ['Employment'] # Change the column to be custom

In [6]:
edd_data

,Employment
industry_title,
Accommodation,26300
Administrative and Support and Waste Ser,94700
"Arts, Entertainment, and Recreation",31300
Construction,85500
Department of Defense,23400
Educational Services,26600
Federal Government excluding Department of Defense,23400
Finance and Insurance,45600
Food Services and Drinking Places,141200


# Grabbing Estimates Data